In [ ]:
# Import Dependencies
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization

In [ ]:
# Set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change Directory
%cd /content/drive/MyDrive/pet_expressions_data

/content/drive/MyDrive/pet_expressions_data


In [ ]:
# Define the path to the AUGMENTED dataset folders
happy_folder = "/content/drive/MyDrive/pet_expressions_data/happy"
sad_folder = "/content/drive/MyDrive/pet_expressions_data/sad"
angry_folder = "/content/drive/MyDrive/pet_expressions_data/angry"
other_folder = "/content/drive/MyDrive/pet_expressions_data/other"

In [ ]:
# Check the number of pictures in each 'emotion' folder, ideally we would want this to be equal
# Define function to count picturues within a folder, takes path as input

def count_pics(folder_path):
    picture_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    picture_count = 0

    # Check if the provided folder path exists
    if not os.path.exists(folder_path):
        print("Folder path does not exist.")
        return 0

    # Get a list of files in the folder
    files = os.listdir(folder_path)

    # Count pictures with valid extensions
    for file in files:
        _, extension = os.path.splitext(file)
        if extension.lower() in picture_extensions:
            picture_count += 1

    return picture_count

In [ ]:
# Happy
happy_picture_count = count_pics(happy_folder)
print(f"Number of pictures in the HAPPY folder: {happy_picture_count}")

Number of pictures in the HAPPY folder: 312


In [ ]:
# Sad
sad_picture_count = count_pics(sad_folder)
print(f"Number of pictures in the SAD folder: {sad_picture_count}")

Number of pictures in the SAD folder: 305


In [ ]:
# Angry
angry_picture_count = count_pics(angry_folder)
print(f"Number of pictures in the ANGRY folder: {angry_picture_count}")

Number of pictures in the ANGRY folder: 294


In [ ]:
 # Other
 other_picture_count = count_pics(other_folder)
print(f"Number of pictures in the  OTHER folder: {other_picture_count}")

Number of pictures in the  OTHER folder: 259


In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

In [ ]:
# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)
other_images = load_images_from_folder(other_folder)

In [ ]:
# Shuffle images in each folder


In [ ]:
# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)
other_labels = [3] * len(other_images)

In [ ]:
# Concatenate images and labels
x = np.array(happy_images + sad_images + angry_images + other_images)
y = np.array(happy_labels + sad_labels + angry_labels + other_labels)

In [ ]:
# Normalize pixel values to range [0, 1]
x = x.astype('float32') / 255.0

In [ ]:
# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, 4)

In [ ]:
# Split the data into 2 sets, a 'main' set and then our test set.
x_main, x_test, y_main, y_test = train_test_split(x, y, test_size=.25, random_state=100)

In [ ]:
# Now splitting our 'main' set into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_main,y_main, random_state=100)

In [ ]:
# Build the CNN model
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))


In [ ]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       7

In [ ]:
# Compile the model with class weights
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [ ]:
# Add early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]),
                 1: total_samples / np.sum(y_train[:, 1]),
                 2: total_samples / np.sum(y_train[:, 2]),
                 3: total_samples / np.sum(y_train[:, 3])}

In [ ]:
# Train/Fit the model with class weights
history = model.fit(x_train.reshape(-1, 48, 48, 1), y_train, batch_size=32, epochs=50, validation_split=0.1, class_weight=class_weights,verbose=0)

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(x_test.reshape(-1, 48, 48, 1), y_test)
losstr, accuracytr = model.evaluate(x_train.reshape(-1, 48, 48, 1), y_train)

30/30 [==============================] - 11s 374ms/step - loss: 1.3828 - accuracy: 0.2778


In [ ]:
print(f"Test accuracy: {accuracy*100:.2f}%")
print(f"Train accuracy: {accuracytr*100:.2f}%")

Test accuracy: 32.05%
Train accuracy: 27.78%


In [ ]:
# Save the trained model
#model.save("pet_expressions_model_Sequential.h5")

In [ ]:
# Create loss plot
import plotly.graph_objects as go

epochs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

# Create the line plot
fig = go.Figure(data=go.Scatter(x=epochs,y=history.history['loss'], mode='lines+markers'))

# Update layout
fig.update_layout(title='Loss Plot',
                  xaxis_title='Epochs',
                  yaxis_title='Loss')

# Show the plot
fig.show()

In [ ]:
# Create accuracy plot
import plotly.graph_objects as go

# Create the line plot
fig = go.Figure(data=go.Scatter(x=epochs,y=history.history['accuracy'], mode='lines+markers'))

# Update layout
fig.update_layout(title='Accuracy Plot',
                  xaxis_title='Epochs',
                  yaxis_title='Accuracy')

# Show the plot
fig.show()

In [ ]:
##################################################################################
#Deploy Model

In [1]:
# from keras.models import load_model
# # Load the saved model
# loaded_model = load_model('pet_expressions_model.h5')

In [ ]:
# Function to load and preprocess images
# def load_images_from_folder(folder):
#     images = []
#     for filename in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder, filename))
#         if img is not None:
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
#             images.append(img)
#     return images

In [ ]:
# # Load a custom test image
# custom_test_image_path = ""

# custom_test_image = cv2.imread(custom_test_image_path)
# custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
# custom_test_image = cv2.resize(custom_test_image, (48, 48))
# custom_test_image = custom_test_image.astype('float32') / 255.0

In [ ]:
# # Reshape the image to match the model input shape
# custom_test_image = np.expand_dims(custom_test_image, axis=0)
# custom_test_image = np.expand_dims(custom_test_image, axis=-1)

In [ ]:
# # Make predictions on the custom test image
# prediction = loaded_model.predict(custom_test_image)
# prediction_prob = prediction[0]

In [ ]:
# emotion_label = np.argmax(prediction[0])

In [ ]:
# Map the predicted label to emotion class
# emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry', 3: 'feeling some type of way...'}
# predicted_emotion = emotion_classes[emotion_label]

In [ ]:
# # Print the custom test image and its predicted label
# print(f"Predicted Emotion: {predicted_emotion}")
# print(f"Confidence [happy, sad, angry,feeling some type of way...]: {prediction_prob}")

In [ ]:
# import matplotlib.pyplot as plt

# #Display the custom test image using matplotlib
# plt.imshow(custom_test_image[0, :, :, 0])
# plt.title(f"Predicted Emotion: {predicted_emotion}")
# plt.axis('off')  # Hide axes
# plt.show()

In [ ]:
# from PIL import Image
# # Display the original custom test image using PIL
# img_pil = Image.open(custom_test_image_path)
# plt.imshow(np.array(img_pil))
# plt.title(f"Predicted Emotion: {predicted_emotion}")
# plt.axis('off')  # Hide axes
# plt.show()